<a href="https://colab.research.google.com/github/rho-selynn/592-HW3/blob/main/592_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torch import int64
import cv2 
import os 
import glob 
from google.colab.patches import cv2_imshow
import numpy as np
import torch
img_dir = "Images"
data_path = os.path.join(img_dir,'*g') 
files = glob.glob(data_path) 
Images = [] 
for f1 in files: 
    img = cv2.imread(f1, 0) 
    Images.append(img)
Images = np.array(Images)
Images = (np.divide(Images, 255))
Images = Images.astype(float)

In [2]:
controlData = np.loadtxt('Robot Log/robot_log.csv', delimiter = ';', skiprows = 1, usecols = (1,2), dtype = float)
print(np.max(controlData[:,0]))
controlData[:,0] /= 15

15.0


In [3]:
from torch.utils.data import TensorDataset
ImagesTrain = torch.tensor(Images[0:799])
steerDataTrain = torch.tensor(controlData[0:799, 0])
trainSteerDataset = TensorDataset(ImagesTrain, steerDataTrain)
ImagesTest = torch.tensor(Images[800:999])
steerDataTest = torch.tensor(controlData[800:999, 0])
testSteerDataset = TensorDataset(ImagesTest, steerDataTest)
throttleDataTrain = torch.tensor(controlData[0:799, 1])
trainThrottleDataset = TensorDataset(ImagesTrain, throttleDataTrain)
throttleDataTest = torch.tensor(controlData[800:999, 1])
testThrottleDataset = TensorDataset(ImagesTest, throttleDataTest)

In [4]:
steerTrainDataLoader = torch.utils.data.DataLoader(trainSteerDataset, batch_size=128, shuffle=True)
steerTestDataLoader = torch.utils.data.DataLoader(testSteerDataset, batch_size=128, shuffle=False)
throttleTrainDataLoader = torch.utils.data.DataLoader(trainThrottleDataset, batch_size=128, shuffle=True)
throttleTestDataLoader = torch.utils.data.DataLoader(testThrottleDataset, batch_size=128, shuffle=False)

In [5]:
class LR(torch.nn.Module):
    def __init__(self):
        super(LR, self).__init__()
        self.linear1 = torch.nn.Linear(160*320, 128) # W:784x128 , b:128x1 , parameters = [W,b]
        self.linear2 = torch.nn.Linear(128, 10)
        self.linear3 = torch.nn.Linear(10, 5)
        self.linear4 = torch.nn.Linear(5, 1)
    
    def forward(self, x):
        x = x.view(-1,160*320)
        transformed_x1 = self.linear1(x)
        transformed_x2 = self.linear2(transformed_x1)
        transformed_x3 = self.linear3(transformed_x2)
        transformed_x4 = self.linear4(transformed_x3)
        return transformed_x4

class LRthrottle(torch.nn.Module):
    def __init__(self):
        super(LRthrottle, self).__init__()
        self.linear1 = torch.nn.Linear(160*320, 128) # W:784x128 , b:128x1 , parameters = [W,b]
        self.linear2 = torch.nn.Linear(128, 10)
        self.linear3 = torch.nn.Linear(10, 5)
        self.linear4 = torch.nn.Linear(5, 1)
    
    def forward(self, x):
        x = x.view(-1,160*320)
        transformed_x1 = self.linear1(x)
        transformed_x2 = self.linear2(transformed_x1)
        transformed_x3 = self.linear3(transformed_x2)
        transformed_x4 = self.linear4(transformed_x3)
        return transformed_x4

steerNet = LR()
steerOptimizer = torch.optim.Adam(steerNet.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()
criterionThrottle = torch.nn.L1Loss()
throttleNet = LRthrottle()
throttleOptimizer = torch.optim.Adam(throttleNet.parameters(), lr=0.01)

In [7]:
#steering trainer
train_loss_history = []
test_loss_history = []
for epoch in range(20):
    train_loss = 0.0
    test_loss = 0.0
    for i, data in enumerate(steerTrainDataLoader):
        images, labels = data
        images = images.to(torch.float32)
        labels = labels.to(torch.float32)
        steerOptimizer.zero_grad()
        predicted_output = steerNet(images.float())
        predicted_output = predicted_output.to(torch.float32)
        loss = criterion(predicted_output, labels)
        loss.backward()
        steerOptimizer.step()
        train_loss += loss.item()
    for i, data in enumerate(steerTestDataLoader):
        with torch.no_grad():
            images, labels = data
            predicted_output = steerNet(images.float())
            loss = criterion(predicted_output, labels.long())
            test_loss += loss.item()
    train_loss = train_loss/len(steerTrainDataLoader)
    test_loss = test_loss/len(steerTestDataLoader)
    train_loss_history.append(train_loss)
    test_loss_history.append(test_loss)
    print('Epoch %s finished with train loss %s and test loss %s'%(epoch, train_loss, test_loss))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([31])) that is different to the input size (torch.Size([31, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([71])) that is different to the input size (torch.Size([71, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, 

Epoch 0 finished with train loss 211.74603288514274 and test loss 0.9688172042369843
Epoch 1 finished with train loss 50.695056097848074 and test loss 89.06995010375977
Epoch 2 finished with train loss 62.12545204162598 and test loss 16.418571949005127
Epoch 3 finished with train loss 5.179111250809261 and test loss 2.4891035556793213
Epoch 4 finished with train loss 6.585627283368792 and test loss 8.872553825378418
Epoch 5 finished with train loss 6.441149881907871 and test loss 2.405965566635132
Epoch 6 finished with train loss 0.9646330731255668 and test loss 0.07136337272822857
Epoch 7 finished with train loss 0.6381242999008724 and test loss 0.7854233384132385
Epoch 8 finished with train loss 0.8701471941811698 and test loss 0.3078116923570633
Epoch 9 finished with train loss 0.31679497233458925 and test loss 0.03206150885671377
Epoch 10 finished with train loss 0.2179463505744934 and test loss 0.13665828853845596
Epoch 11 finished with train loss 0.2548051050731114 and test loss 

In [8]:
#throttle trainer
train_loss_history = []
test_loss_history = []
for epoch in range(40):
    train_loss = 0.0
    test_loss = 0.0
    for i, data in enumerate(throttleTrainDataLoader):
        images, labels = data
        images = images.to(torch.float32)
        labels = labels.to(torch.float32)
        throttleOptimizer.zero_grad()
        predicted_output = throttleNet(images.float())
        predicted_output = predicted_output.to(torch.float32)
        loss = criterion(predicted_output, labels)
        loss.backward()
        throttleOptimizer.step()
        train_loss += loss.item()
    for i, data in enumerate(throttleTestDataLoader):
        with torch.no_grad():
            images, labels = data
            predicted_output = throttleNet(images.float())
            loss = criterion(predicted_output, labels.long())
            test_loss += loss.item()
    train_loss = train_loss/len(throttleTrainDataLoader)
    test_loss = test_loss/len(throttleTestDataLoader)
    train_loss_history.append(train_loss)
    test_loss_history.append(test_loss)
    print('Epoch %s finished with train loss %s and test loss %s'%(epoch, train_loss, test_loss))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([31])) that is different to the input size (torch.Size([31, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([71])) that is different to the input size (torch.Size([71, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, 

Epoch 0 finished with train loss 13307.067881230798 and test loss 6497.4248046875
Epoch 1 finished with train loss 2926.810812813895 and test loss 2226.5338134765625
Epoch 2 finished with train loss 1314.0075054168701 and test loss 1244.5053100585938
Epoch 3 finished with train loss 454.61235870633806 and test loss 357.4967041015625
Epoch 4 finished with train loss 196.5564176504101 and test loss 19.102609634399414
Epoch 5 finished with train loss 82.11539105006626 and test loss 29.217647552490234
Epoch 6 finished with train loss 14.71764476810183 and test loss 33.36130237579346
Epoch 7 finished with train loss 21.31222861153739 and test loss 0.4587346762418747
Epoch 8 finished with train loss 5.31364957349641 and test loss 8.591838836669922
Epoch 9 finished with train loss 3.211380170924323 and test loss 0.9211789667606354
Epoch 10 finished with train loss 2.2199839864458357 and test loss 1.3541875183582306
Epoch 11 finished with train loss 0.5923635278429303 and test loss 0.942280024

In [9]:
from torch import int64
import cv2 
import os 
import glob 
from google.colab.patches import cv2_imshow
import numpy as np
import torch
img_dir = "Images"
data_path = os.path.join(img_dir,'*g') 
files = glob.glob(data_path) 
Images = [] 
for f1 in files: 
    img = cv2.imread(f1, 0) 
    Images.append(img)
Images = np.array(Images)
Images = (np.divide(Images, 255))
Images = Images.astype(float)
Images = torch.tensor(Images)

for i in np.linspace(1, 999, 100).astype(int):
  predicted_output_steer = steerNet(Images[i].float())
  predicted_output_throttle = throttleNet(Images.float()[i])
  correct_steer = controlData[i,0]
  correct_throttle = controlData[i,1]
  print ("correct steer: " + str(correct_steer) + " predicted steer: " + str(predicted_output_steer))
  print ("correct throttle: " + str(correct_throttle) + " predicted throttle: " + str(predicted_output_throttle))

correct steer: 0.0 predicted steer: tensor([[0.1019]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([[0.4683]], grad_fn=<AddmmBackward0>)
correct steer: 0.0 predicted steer: tensor([[-0.1900]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([[0.4041]], grad_fn=<AddmmBackward0>)
correct steer: -0.7830426666666667 predicted steer: tensor([[0.0228]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([[0.5004]], grad_fn=<AddmmBackward0>)
correct steer: -1.0 predicted steer: tensor([[-0.0864]], grad_fn=<AddmmBackward0>)
correct throttle: 0.0 predicted throttle: tensor([[0.4291]], grad_fn=<AddmmBackward0>)
correct steer: 0.6324920666666666 predicted steer: tensor([[-0.0864]], grad_fn=<AddmmBackward0>)
correct throttle: 1.0 predicted throttle: tensor([[0.4291]], grad_fn=<AddmmBackward0>)
correct steer: 0.35304953333333333 predicted steer: tensor([[0.1162]], grad_fn=<AddmmBackward0>)
correct throttle: 1.0 predic